#  Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Set plot style for better aesthetics

In [2]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 7)

# Load the Data using NumPy

In [4]:
try:
    datos_precios = np.loadtxt(
        "manzanas.csv",      
        delimiter=",",       
        skiprows=1,          
        usecols=np.arange(1, 85) 
    )
    print("Data loaded successfully!")
except IOError:
    print("Error: 'manzanas.csv' not found. Please make sure the file is in the same directory.")
    # In case of error, create dummy data to allow the rest of the script to run
    datos_precios = np.random.rand(5, 84) * 100 + 50

Data loaded successfully!
